<a href="https://colab.research.google.com/github/goncaloneto13/Face-Recognition-using-VGGFace2-and-Linear-SVM-CACD-/blob/main/Reconhecedor_VGGFace2_e_LinerSVM(CACD)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os
import math
import tensorflow as tf
import numpy  as  np
import matplotlib.pyplot as plt
 
%tensorflow_version 1.x
from google.colab import drive
from keras.models import Model
from keras.layers import Input, Conv2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
!pip install keras_vggface
!pip install keras_applications
!pip install keras_preprocessing
from keras_vggface.vggface import VGGFace
from sklearn.neighbors import KNeighborsClassifier
from skimage.exposure import equalize_hist
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
import numpy as np
from keras.preprocessing import image
from keras_vggface import utils
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
import io
from skimage.feature import hog
drive.mount('/content/drive')
pasta = 'drive/My Drive/CACD_VS'
import sklearn.metrics as metrics
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from keras.optimizers import SGD
import numpy
from keras.preprocessing.image import ImageDataGenerator
 
osOrd = sorted(os.listdir(pasta))
datagen=ImageDataGenerator()

TensorFlow is already loaded. Please restart the runtime to change versions.
     |████████████████████████████████| 51kB 2.8MB/s 
Mounted at /content/drive


In [ ]:
ext_model = VGGFace(model='senet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

104947712/104944616 [==============================] - 2s 0us/step


In [ ]:
def retirar_descritor(n):
  img = image.load_img(pasta + '/' + osOrd[n],target_size=(224,224))
  x_train = image.img_to_array(img)
  x_train = np.expand_dims(x_train, axis=0)
  x_train = utils.preprocess_input(x_train, version=2)
  print('.', end ='')
  descritor = ext_model.predict(x_train,verbose=0)
  return descritor.reshape(descritor.shape[1])

arquivo = open('drive/My Drive/descritores.txt','w')

descritores = []
for i in range(0,8000):
  arquivo.write(str(retirar_descritor(i))+'\n')
  descritores.append(retirar_descritor(i))


..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
np.savetxt('drive/My Drive/descritoresVs.txt',np.array(descritores))
 
'''
arquivo = open('drive/My Drive/descritores.txt','r')
descritores = [] 
for l in arquivo:
  descritores.append(np.array(l))
  #np.asarray(x, dtype='float64')
'''

In [ ]:
descritores = np.loadtxt('drive/My Drive/PIBIC/descritoresVs.txt')

In [ ]:
tam = 8000
dist_pares = []
m_scores = 0
print(len(descritores))
for i in range(0,tam,2):
  print('.',end='') 
  des1 = descritores[i]
  des2 = descritores[i+1]
  distancia = cosine(des1, des2)
  dist_pares.append([distancia])

x_treino = []
y_treino = []
x_teste = []
y_teste = []

n=0
ini = 0
fim = ini + 400

for i in range(0,int(tam/2)):
  if i >= ini and i < fim:
    #print(i, 'teste')
    x_teste.append(dist_pares[i])
    if i < ini+200:
      y_teste.append(1)
    else:
      y_teste.append(0)
  else:
    #print(i, 'treino')
    x_treino.append(dist_pares[i])
    if n == 400:
      n=0
    if n < 200:
      y_treino.append(1)
    else:
      y_treino.append(0) 
    n += 1  

8000
...................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
acertos = 0
#clf = LinearSVC()
#clf = SVC()
#lda = LDA()
#lda.fit(x_treino,y_treino)
scores = []
clf = SVC(kernel='linear', probability=True)
clf.fit(x_treino,y_treino)

for i in range(0,400):
  
  classe  = clf.predict(np.array(x_teste[i]).reshape(1, -1))
  scores.append(clf.decision_function(np.array(x_teste[i]).reshape(1, -1)))
 # classe = lda.predict(np.array(x_teste[i]).reshape(1, -1))
  print('imagem: ', y_teste[i], ' : ', classe)
  if classe == y_teste[i]:
    acertos += 1

print('acertou', acertos,' de ', 400, '  ', acertos/400*100,'%')    
#print(x_treino)

#plt.scatter(x_treino,y_treino)
print(clf.score(x_teste, y_teste))
#scores = clf.decision_function(x_teste)
#print(scores)

In [ ]:
vgg_face = VGGFace(model='senet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

for l in vgg_face.layers[:-7]:
    l.trainable=False

d_1 = Dense(4096, activation="relu",name = 'd_1')(vgg_face.output)
#d_2 = Dense(4096, activation="relu",name = 'd_2')(d_1)
#d_3 = Dense(4096, activation="relu",name = 'd_3')(d_2)
newlayer=Dense(200, activation='softmax')(d_1)

ft_model = Model(inputs=vgg_face.input, outputs=newlayer)
ft_model.compile(optimizer=SGD(lr=0.001,decay=1e-6,momentum=0.9, nesterov=True),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
ft_model.summary()

In [ ]:
x_treino = []
y_treino = []
x_teste = []
y_teste = []

for i in range(0,400,2):
  x_treino.append(np.array(image.load_img(pasta + '/' + osOrd[i],target_size=(224,224))))
  y_treino.append(np.uint8(int (osOrd[i][:4])))

x_treino = np.array(x_treino)
x_treino = x_treino/255

y_treino = np.array(y_treino)
y_treino = tf.keras.utils.to_categorical(y_treino, 200) 

In [ ]:
train_iterator=datagen.flow(x_treino,y_treino,batch_size=32,shuffle=True)
#test_iterator =datagen.flow(x_teste ,y_teste ,batch_size=32,shuffle=False)

train_metrics = ft_model.evaluate_generator(train_iterator,verbose=1)
#test_metrics = ft_model.evaluate_generator(test_iterator,verbose=1)
#print('Antes(train_set):',train_metrics)
#print('Antes(test_set): ',test_metrics)

# treinamento (ajuste fino das camadas treináveis)
#history = ext_model.fit_generator(generator=train_iterator,validation_data=test_iterator,epochs=80,verbose=1)
history = ft_model.fit_generator(generator=train_iterator,epochs=12,verbose=1)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['loss'])
plt.ylabel('Accuracy')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
tf_model = Model(inputs= ft_model.input,outputs=ft_model.get_layer('d_1').output)

Modelo VGGFace2 para extração de caracteristicas 


Retira os descritores

Calcula a distancia de cada par de imagem e salva nos vetores classificados como 1 para distancia de pares positivos e 0 para distancia de pares negativos 

As 400 primeiras imagens são de pares positivos e as proximas 400 imagens de pares negativos 

Ajusta o classificador LinearSVM com modelos de treino 

acertou 352  de  400    88.0 %  0.88 com tf 2048\
acertou 354  de  400    88.5 %  0.885 co tf 4096\
acertou 358  de  400    89.5 %  0.895 com tf dense_2\
acertou 356  de  400    89.0 %  0.89 com ext

Curva ROC\

https://bcsiriuschen.github.io/CARC/

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_teste, scores)
roc_auc = metrics.roc_auc_score(y_teste, scores)
#plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',lw=lw, label='VGGFace2 + SVM (area = %0.2f)' % roc_auc)
plt.xlim([0.0, 1.00])
plt.ylim([0.3, 1.001])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve on CACD-VS')
plt.legend(loc="lower right")
plt.rcParams['figure.figsize'] = (8.3,7.0)
plt.show()

In [ ]:
tam = 8000
dist_pares = []
m_scores = 0
scores = []
y_testet = []


for i in range(0,tam,2):
  print('.',end='') 
  des1 = descritores[i]
  des2 = descritores[i+1]
  distancia = cosine(des1, des2)
  dist_pares.append(distancia)
print()

for p in range(0,4000,400):
  x_treino = []
  y_treino = []
  x_teste = []
  y_teste = []


  n=0
  ini = p
  fim = ini + 400

  for i in range(int(tam/2)):
    if i >= ini and i < fim:
     # print(i, 'teste', end = '  ')
      x_teste.append([dist_pares[i]])
      if i < ini+200:
        y_teste.append(1)
      else:
        y_teste.append(0)
    else:
    # print(i, 'treino', end='  ')
      x_treino.append([dist_pares[i]])
      if n == 400:
        n=0
      if n < 200:
        y_treino.append(1)
      else:
        y_treino.append(0) 
      n += 1  

  acertos = 0
  clf = LinearSVC(max_iter=44)
  #clf = SVC()
  clf.fit(x_treino,y_treino)

  for i in range(0,400):
    classe  = clf.predict(np.array(x_teste[i]).reshape(-1, 1))
    scores.append(clf.decision_function(np.array(x_teste[i]).reshape(1, -1)))
    #print('imagem: ', y_teste[i], ' : ', classe)
    if classe == y_teste[i]:
      acertos += 1
  y_testet += y_teste
  scor = clf.score(x_teste, y_teste)
  m_scores += scor
  print(int(p/400)+1, ' - acertou', acertos,' de 400 - ', scor)

print(m_scores/10)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_testet, scores)
roc_auc = metrics.roc_auc_score(y_testet, scores)
#plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',lw=lw, label='VGGFace2 + SVM (area = %0.2f)' % roc_auc)
plt.xlim([0.0, 1.00])
plt.ylim([0.3, 1.001])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve on CACD-VS')
plt.legend(loc="lower right")
plt.rcParams['figure.figsize'] = (8.4,7.0)
plt.show()

1  - acertou 354  de 400 -  0.885\
2  - acertou 342  de 400 -  0.855\
3  - acertou 334  de 400 -  0.835\
4  - acertou 352  de 400 -  0.88\
5  - acertou 356  de 400 -  0.89\
6  - acertou 344  de 400 -  0.86\
7  - acertou 335  de 400 -  0.8375\
8  - acertou 341  de 400 -  0.8525\
9  - acertou 346  de 400 -  0.865\
10  - acertou 346  de 400 -  0.865\
0.8625


'''
for i in range(0,400,2):
  x_teste.append(np.array(image.load_img(pasta + '/' + osOrd[i+1],target_size=(224,224))))
  y_teste.append(np.uint8(int (osOrd[i+1][:4])))

x_teste = np.array(x_teste)
x_teste = x_teste/255

y_teste = np.array(y_teste)
y_teste = tf.keras.utils.to_categorical(y_teste, 200) 
'''